## Integrantes:
1. Camila Coltriani
2. Luis Dartayet
3. Irania Fuentes
4. Jonathan Fichelson
5. Ornella Cevoli
# Trabajo práctico  4: Analisis de los sentimientos en Twitter

Intro: Las redes sociales como Twitter han demostrado ser excelentes recursos de información sobre muchos eventos que acontecen en el mundo. Tiene el poder de cambiar las opiniones de millones de personas siendo especialmente útil para influir en las masas: campañas políticas, cotización de monedas virtuales, publicidad de ventas, entre otros.



## Objetivo:
El análisis de los sentimientos se puede utilizar para predecir el comportamiento de personas y propagar cambios en tiempo real a medida que se desarrolla el evento que se quiere estudiar.

_______________________________________

In [1]:
# # Librerías
## import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
## from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
# from sklearn.metrics import accuracy_score,plot_confusion_matrix,roc_auc_score, classification_report, confusion_matrix, precision_recall_curve, auc
# from sklearn.naive_bayes import GaussianNB
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn import tree
# from scipy.stats import mode
# import seaborn as sns
## import re

# # nltk
# import nltk
# from nltk.tokenize import word_tokenize, sent_tokenize
## from nltk.corpus import stopwords
## from  nltk.stem import SnowballStemmer
## from nltk.stem import WordNetLemmatizer
## from sklearn.feature_extraction.text import CountVectorizer,
# TfidfTransformer
# from sklearn.feature_extraction.text import TfidfVectorizer


# from sklearn.naive_bayes import MultinomialNB
# from sklearn.model_selection import GridSearchCV,StratifiedKFold,train_test_split
# from sklearn.metrics import accuracy_score
## from sklearn.pipeline import Pipeline

# from sklearn.pipeline import make_pipeline, make_union
# from sklearn.preprocessing import StandardScaler
# from sklearn.impute import SimpleImputer
## from sklearn.base import BaseEstimator, TransformerMixin




## Importación

BAJAR DATASET DE KAGGLE, RENOMBRARLO COMO TWITTER Y PONERLO EN LA CARPETA DATA

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('./data/twitter.csv', encoding='ANSI')
data.columns = [ 'target', 'id', 'date', 'flag', 'user', 'text']
print(data.shape)
data.head()

(1599999, 6)


,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [4]:
# Check nulls and duplicates
print('Nulls: ', data.isnull().sum())
print('Duplicates: ', data.duplicated().sum())

Nulls:  target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64
Duplicates:  0


In [5]:
# Remove useless columns
data.drop(['id', 'date', 'flag', 'user'], axis=1, inplace=True)

In [6]:
# Rename target
data['target'] = data['target'].map({0: 'negative', 4: 'positive'})

In [7]:
# Check target distribution
print('Target distribution: ', data.target.value_counts())

Target distribution:  positive    800000
negative    799999
Name: target, dtype: int64


In [8]:
#Asignamos las variables X e Y a modelar
X = data.text
y = data.target

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
#Dividimos en train-test
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y, test_size=0.3)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1119999,)
(480000,)
(1119999,)
(480000,)


In [11]:
import re

from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

In [12]:
# Expresion regular para eliminar del corpus signos de puntación/ @/ direcciones electronicas #numeros
limpieza_re = "\d+[^0-9]|@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]"

stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

#Funcion para limpiar el corpus
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(limpieza_re, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)



In [13]:
from sklearn.base import BaseEstimator, TransformerMixin

In [14]:
class TextProcessing(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return X.apply(preprocess)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer


In [16]:
common_steps = [
            ('text_processing', TextProcessing()),
            ('vectorizer', CountVectorizer()),
            ]


In [17]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV,StratifiedKFold

In [18]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import tree

In [19]:


MNB_pipe = [Pipeline( common_steps + [('mnb', MultinomialNB())]), {'mnb__alpha':[1, 10, 20]}]
KNN_pipe = [Pipeline( common_steps + [('knn', KNeighborsClassifier())]), {'knn__n_neighbors':[5]}]
LR_pipe = [Pipeline( common_steps + [('lr', LogisticRegression())]), {'lr__C':[0.1,1,10], 'lr__penalty':['l1','l2']}]
DT_pipe = [Pipeline( common_steps + [('dt', tree.DecisionTreeClassifier())]), {'dt__max_depth':[2, 4, 8] }]

skf=StratifiedKFold(n_splits=3,random_state=0,shuffle=True)
models = []


In [20]:
pipelines = [ LR_pipe]

for pipe in pipelines:
    GS_CV=GridSearchCV(pipe[0],pipe[1],cv=skf,verbose=10,n_jobs=3);
    GS_CV.fit(X_train, y_train);
    models.append(GS_CV)
    print('best score:',GS_CV.best_score_)
    print('best params:',GS_CV.best_params_) 

Fitting 3 folds for each of 6 candidates, totalling 18 fits


c:\Users\LuisD\anaconda3\envs\dh\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
9 fits failed out of a total of 18.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\LuisD\anaconda3\envs\dh\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\LuisD\anaconda3\envs\dh\lib\site-packages\sklearn\pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "c:\Users\LuisD\anaconda3\envs\dh\lib\site-packages\sklearn\linear_model\_logistic.py", line 1091, in fit
    solver = _check_solv

best score: 0.7751256920765108
best params: {'lr__C': 0.1, 'lr__penalty': 'l2'}


c:\Users\LuisD\anaconda3\envs\dh\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [21]:
pipelines = [ DT_pipe]

for pipe in pipelines:
    GS_CV=GridSearchCV(pipe[0],pipe[1],cv=skf,verbose=10,n_jobs=3);
    GS_CV.fit(X_train, y_train);
    models.append(GS_CV)
    print('best score:',GS_CV.best_score_)
    print('best params:',GS_CV.best_params_) 

Fitting 3 folds for each of 3 candidates, totalling 9 fits
best score: 0.5626638952356208
best params: {'dt__max_depth': 8}


In [22]:
pipelines = [ MNB_pipe]

for pipe in pipelines:
    GS_CV=GridSearchCV(pipe[0],pipe[1],cv=skf,verbose=10,n_jobs=3);
    GS_CV.fit(X_train, y_train);
    models.append(GS_CV)
    print('best score:',GS_CV.best_score_)
    print('best params:',GS_CV.best_params_) 

Fitting 3 folds for each of 3 candidates, totalling 9 fits
best score: 0.7662113984030343
best params: {'mnb__alpha': 10}


In [23]:
# pipelines = [ KNN_pipe]

# for pipe in pipelines:
#     GS_CV=GridSearchCV(pipe[0],pipe[1],cv=skf,verbose=10,n_jobs=3);
#     GS_CV.fit(X_train, y_train);
#     models.append(GS_CV)
#     print('best score:',GS_CV.best_score_)
#     print('best params:',GS_CV.best_params_) 

Fitting 3 folds for each of 1 candidates, totalling 3 fits


: 

In [28]:
models[0].predict(X_test)

array(['negative', 'positive', 'negative', ..., 'positive', 'positive',
       'negative'], dtype=object)

In [30]:
## Evaluación del modelo

from sklearn.metrics import accuracy_score,plot_confusion_matrix,roc_auc_score, classification_report, confusion_matrix, precision_recall_curve, auc

y_pred = models[0].predict(X_test)
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Classification report: ', classification_report(y_test, y_pred))
print('Confusion matrix: ', confusion_matrix(y_test, y_pred))


Accuracy:  0.7765958333333334
Classification report:                precision    recall  f1-score   support

    negative       0.79      0.75      0.77    240000
    positive       0.76      0.81      0.78    240000

    accuracy                           0.78    480000
   macro avg       0.78      0.78      0.78    480000
weighted avg       0.78      0.78      0.78    480000

Confusion matrix:  [[179404  60596]
 [ 46638 193362]]


In [40]:
# Predict new data

new_data = ['supercalifragilisticexpialidocious']

new_data = pd.Series(new_data)
new_data = new_data.apply(preprocess)

models[0].predict(new_data)



array(['positive'], dtype=object)

Configuración del Pipeline

PARA LEER: el primer pipe es con base a la clase 47_1 clasificacion de textos
pipeline para varios modelos viene de aca https://www.youtube.com/watch?v=ES0lM7QBZnI&list=FLrNGJ7KGOB1XGjvk9OvahOw&index=1
que está buena la idea, y lo que se haria luego es usar CV y tunear los hiperparametros sobre el mejor modelo. 

In [62]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\LuisD\AppData\Roaming\nltk_data...


True

In [63]:
sentimentIntensityAnalyzer = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = sentimentIntensityAnalyzer.polarity_scores(sentence)
    return score

data['sentiment'] = data['text'].apply(sentiment_analyzer_scores)
data.head()

,target,text,sentiment
0,negative,is upset that he can't update his Facebook by ...,"{'neg': 0.303, 'neu': 0.697, 'pos': 0.0, 'comp..."
1,negative,@Kenichan I dived many times for the ball. Man...,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'comp..."
2,negative,my whole body feels itchy and like its on fire,"{'neg': 0.321, 'neu': 0.5, 'pos': 0.179, 'comp..."
3,negative,"@nationwideclass no, it's not behaving at all....","{'neg': 0.241, 'neu': 0.759, 'pos': 0.0, 'comp..."
4,negative,@Kwesidei not the whole crew,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


In [ ]:
#armar un pipeline que incluya la verctorizacion y el clasificador para
#optimizar sobre todos los hiperparámetros a la vez.

#ttps://www.kaggle.com/code/balatmak/text-preprocessing-steps-and-universal-pipeline/notebook 

#https://www.kaggle.com/code/balatmak/text-classification-pipeline-newsgroups20/notebook esta notebook está buenisima

In [ ]:
#ESTO ES UNA BASE 

#iniciamos el modelo
#MNB = MultinomialNB()

In [ ]:
# text_tfidf = Pipeline([('vect', CountVectorizer(stop_words='english')), ('tfidf', TfidfTransformer())])
# tfidf_matrix = text_tfidf.fit_transform(texts_all_train)
#https://www.kaggle.com/code/benboyet/web-mining-project-final-prediction/script

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pipe_pred = GS_CV.predict(X_test)

conf_matrix = confusion_matrix(y_test, pipe_pred); 
plt.figure(figsize=(5, 2)); sns.heatmap(conf_matrix,  annot=True, fmt="d");
plt.title("Confusion matrix"); plt.ylabel('True class'); plt.xlabel('Predicted class');plt.show()

In [ ]:
from sklearn.metrics import  classification_report

In [ ]:
#reporte de clasificación
print(classification_report(y_test, pipe_pred))

In [ ]:
#modelo final con parametros seteados

vectorizer_2=TfidfVectorizer(stop_words=stop_words,strip_accents='unicode', min_df=1);
MNB = MultinomialNB(alpha=2,)
pasos_2=[('vectorizer',vectorizer_2),('MNB',MNB)]

In [ ]:
#configurar pipe
pipe_2 = Pipeline(pasos_2)
pipe_2.fit(X_train, y_train)

In [ ]:
pipe_2_pred= pipe_2.predict(X_test)

In [ ]:
X_test

In [ ]:
#reporte de clasificación
print(classification_report(y_test, pipe_2_pred))

In [ ]:
conf_matrix = confusion_matrix(y_test, pipe_2_pred); 
plt.figure(figsize=(5, 2)); sns.heatmap(conf_matrix,  annot=True, fmt="d");
plt.title("Confusion matrix"); plt.ylabel('True class'); plt.xlabel('Predicted class');plt.show()

In [ ]:
pipe_2_pred= pipe_2.predict(X_test)

"pipeline para varios modelos"

In [ ]:
pipeline_lr = Pipeline([('vect',vectorizer),
                        ('lr_classifier',LogisticRegression())])

In [ ]:
pipeline_knn = Pipeline([('vect',vectorizer),
                        ('knn_classifier',KNeighborsClassifier())])

In [ ]:
#lista de pipelines
pipelines = [pipeline_lr, pipeline_knn]

In [ ]:
best_accuracy=0.0
best_classifier=0
best_pipeline=""

In [ ]:
#diccionario de pipeline y tipos de clasificador para referencia
pipe_dict = {0: "Logistc Regresion", 1:"knn"}

#entrenamiento 
for pipe in pipelines:
    pipe.fit(X_train, y_train)

In [ ]:
for i, model in enumerate(pipelines):
    print("{} Accuracy para el conjunto de prubea: {}".format(pipe_dict[i], model.score(X_test, y_test)))